In [1]:
import pygsti
from pygsti.extras.interpygate import *

In [2]:
from pygsti.extras.interpygate import do_process_tomography, vec, unvec

In [3]:
from pygsti.tools.basistools import change_basis

In [4]:
from mpi4py import MPI
_comm = MPI.COMM_WORLD
_rank = _comm.Get_rank()
_size = _comm.Get_size()

In [5]:
import numpy as _np
from scipy.linalg import expm as _expm

In [6]:
from pygsti.extras.interpygate.core import PhysicalProcess, InterpolatedDenseOp, InterpolatedOpFactory

In [7]:
class ExampleProcess(PhysicalProcess):
    def __init__(self):
        import numpy as _np
        import scipy as _sp

        self.Hx = _np.array([[0, 0, 0, 0],
                            [0, 0, 0, 0],
                            [0, 0, 0, -1],
                            [0, 0, 1, 0]], dtype='float')
        self.Hy = _np.array([[0, 0, 0, 0],
                            [0, 0, 0, 1],
                            [0, 0, 0, 0],
                            [0, -1, 0, 0]], dtype='float')
        self.Hz = _np.array([[0, 0, 0, 0],
                            [0, 0, -1, 0],
                            [0, 1, 0, 0],
                            [0, 0, 0, 0]], dtype='float')

        self.dephasing_generator = _np.diag([0, -1, -1, 0])
        self.decoherence_generator = _np.diag([0, -1, -1, -1])
        num_params = 6  # omega (0), phase (1), detuning (2), dephasing (3), decoherence (4), time (5)
        process_shape = (4, 4)
        super().__init__(num_params, process_shape, 
                         aux_shape=(),  # a single float
                         num_params_evaluated_as_group=1)  # time values can be evaluated all at once

    
    def advance(self, state, v, times):
        state = _np.array(state, dtype='complex')
        omega, phase, detuning, dephasing, decoherence = v

        H = (omega * _np.cos(phase) * self.Hx + omega * _np.sin(phase) * self.Hy + detuning * self.Hz)
        L = dephasing * self.dephasing_generator + decoherence * self.decoherence_generator

        processes = [change_basis(_expm((H + L) * t),'pp', 'col') for t in times]
        states = [unvec(_np.dot(process, vec(_np.outer(state, state.conj())))) for process in processes]

        return states

    def create_process_matrices(self, v, grouped_v, comm=None):                                                                                                                                                                                             
        print(f'Calling process tomography as {comm.Get_rank()} of {comm.Get_size()} on {comm.Get_name()}.')
        times = grouped_v[0]
        def state_to_process_mxs(state):
            return self.advance(state, v, times)
        processes = do_process_tomography(state_to_process_mxs, #opt_args={'v':v, 'times':times},
                                          n_qubits = 1, basis='pp', time_dependent=True, comm=comm)
        return _np.array(processes) # must return an *array* of appropriate shape
    
    def create_aux_data(self, v, grouped_v, comm=None):
        omega, phase, detuning, dephasing, decoherence = v
        times = grouped_v[0]
        return _np.array([t*omega for t in times], 'd')
    
example_process = ExampleProcess()
target_mx = example_process.create_process_matrices(_np.array([1.0, 0.0, 0.0, 0.0, 0.0]), [[_np.pi/2]], comm=_comm)[0]
target_op = pygsti.obj.StaticDenseOp(target_mx)
print(target_op)
#target_mx[0]

Calling process tomography as 0 of 1 on MPI_COMM_WORLD.
StaticDenseOp with shape (4, 4)
 1.00   0   0   0
   0 1.00   0   0
   0   0   0-1.00
   0   0 1.00   0



In [8]:
param_ranges = ([(0.9, 1.1, 2),  # omega
                 (-.1, .1, 2),   # phase
                 (-.1, .1, 2),   # detuning
                 (0, 0.1, 2),    # dephasing
                 (0, 0.1, 2),    # decoherence
                 _np.linspace(_np.pi / 2, _np.pi / 2 + .5, 10)  # time
                ])
interp_op = InterpolatedDenseOp.create_by_interpolating_physical_process(target_op, example_process, param_ranges, comm=_comm)

Group 0 processing 32 points on 1 processors.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1

In [9]:
# import pickle
# pickle.dump(interp_op, open("test.pkl",'wb'))
# interp_op2 = pickle.load(open("test.pkl",'rb'))
# interp_op2.to_dense()

In [10]:
print(interp_op.to_dense())
interp_op.from_vector([1.1, 0.01, 0.01, 0.055, 0.055, 1.59])
print(interp_op.to_vector())
interp_op.to_dense()

[[ 1.00000000e+00 -8.95622450e-20 -4.48403940e-17  3.54197588e-18]
 [-1.20808066e-19  8.33503770e-01 -1.19063355e-02 -8.30148554e-03]
 [-2.37683733e-18 -5.26857267e-03 -2.34251103e-01 -8.45981132e-01]
 [ 8.58292135e-18  7.77865980e-03  8.45896673e-01 -1.97823301e-01]]
[1.1   0.01  0.01  0.055 0.055 1.59 ]


array([[ 1.00000000e+00, -3.62219581e-17, -1.53401152e-17,
         1.14449925e-17],
       [ 3.58256010e-20,  8.41206474e-01,  3.53082074e-03,
         1.82045502e-02],
       [-1.77042081e-18,  1.36513911e-02, -1.74485238e-01,
        -8.65189315e-01],
       [ 8.77988237e-18,  1.23410808e-03,  8.65308324e-01,
        -1.31042228e-01]])

In [11]:
class TargetOpFactory(pygsti.obj.OpFactory):
    def __init__(self):
        self.process = ExampleProcess()
        pygsti.obj.OpFactory.__init__(self, dim=4, evotype="densitymx")
        
    def create_object(self, args=None, sslbls=None):
        assert(sslbls is None) # don't worry about sslbls for now -- these are for factories that can create gates placed at arbitrary circuit locations
        assert(len(args) == 2)  # t (time), omega
        t, omega = args
        mx = self.process.create_process_matrices(_np.array([omega, 0.0, 0.0, 0.0, 0.0]), [[t]], comm=_comm)[0]
        return pygsti.obj.StaticDenseOp(mx)

arg_ranges = [_np.linspace(_np.pi / 2, _np.pi / 2 + .5, 10),  # time
              (0.9, 1.1, 2)  # omega
             ]

param_ranges = [(-.1, .1, 2),  # phase
                (-.1, .1, 2),  # detuning
                (0, 0.1, 2),   # dephasing
                (0, 0.1, 2)    # decoherence
               ]
arg_indices = [5, 0]  #indices for time and omega within ExampleProcess's parameters (see ExampleProcess.__init__)

opfactory = InterpolatedOpFactory.create_by_interpolating_physical_process(
    TargetOpFactory(), example_process, arg_ranges, param_ranges, arg_indices, comm=_comm)

Group 0 processing 32 points on 1 processors.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling proces

Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling 

Calling process tomography as 0 of 1 on comm_group_0.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Calling process tomography as 0 of 1 on comm_world.
Group 0 processing 32 points on 1 processors.


In [12]:
print(opfactory.num_params)
print(opfactory.to_vector())
opfactory.from_vector(_np.array([0.01, 0.01, 0.055, 0.055]))
op = opfactory.create_op((1.59, 1.1))
print(op.to_vector())
print(op)

4
[0.   0.   0.05 0.05]
Calling process tomography as 0 of 1 on comm_world.
[0.01  0.01  0.055 0.055]
InterpolatedDenseOp with shape (4, 4)
 1.00   0   0   0
   0 0.84   0 0.02
   0 0.01-0.18-0.86
   0   0 0.87-0.13



In [13]:
op.aux_data

array(1.749)

In [14]:
#print(op.target_op)

# Testing `do_process_tomography` subroutine

In [15]:
""" Demonstrate the process tomography function with (potentially) time-dependent outputs. """
sigI = _np.array([[1, 0], [0, 1]], dtype='complex')
sigX = _np.array([[0, 1], [1, 0]], dtype='complex')
sigY = _np.array([[0, -1.j], [1.j, 0]], dtype='complex')
sigZ = _np.array([[1, 0], [0, -1]], dtype='complex')
theta = .32723
u = _np.cos(theta) * sigI + 1.j * _np.sin(theta) * sigX
v = _np.sin(theta) * sigI - 1.j * _np.cos(theta) * sigX

U = _np.kron(u, v)
n_qubits = 2
# U = u
# n_qubits = 1
test_process = _np.kron(U.conj().T, U)


def single_time_test_function(pure_state, test_process=test_process):
    rho = vec(_np.outer(pure_state, pure_state.conj()))
    return unvec(_np.dot(test_process, rho))


def multi_time_test_function(pure_state, test_process=test_process):
    rho = vec(_np.outer(pure_state, pure_state.conj()))
    return [unvec(_np.dot(test_process, rho)), unvec(_np.dot(_np.linalg.matrix_power(test_process,2), rho))]


process_matrix = do_process_tomography(single_time_test_function, n_qubits=2, verbose=False)
if _rank == 0:
    test_process_pp = change_basis(test_process, 'col', 'pp')
    print("\nSingle-time test result should be True:")
    print(_np.isclose(process_matrix, test_process_pp).all())

process_matrices = do_process_tomography(multi_time_test_function, n_qubits=2, verbose=False, time_dependent=True)
if _rank==0:
    test_process = change_basis(test_process, 'col', 'pp')
    print("\nMulti-time test result should be [True, False]:")
    print([_np.isclose(x, test_process).all() for x in process_matrices])


Single-time test result should be True:
True

Multi-time test result should be [True, False]:
[True, False]


#  OLD VERSION

In [16]:
#OLD (REMOVE LATER) 
# class ProcessFunction(object):
#     def __init__(self):
#         import numpy as _np
#         import scipy as _sp

#         self.Hx = _np.array([[0, 0, 0, 0],
#                             [0, 0, 0, 0],
#                             [0, 0, 0, -1],
#                             [0, 0, 1, 0]], dtype='float')
#         self.Hy = _np.array([[0, 0, 0, 0],
#                             [0, 0, 0, 1],
#                             [0, 0, 0, 0],
#                             [0, -1, 0, 0]], dtype='float')
#         self.Hz = _np.array([[0, 0, 0, 0],
#                             [0, 0, -1, 0],
#                             [0, 1, 0, 0],
#                             [0, 0, 0, 0]], dtype='float')

#         self.dephasing_generator = _np.diag([0, -1, -1, 0])
#         self.decoherence_generator = _np.diag([0, -1, -1, -1])

#     def advance(self, state, v = None, times = None):
#         state = _np.array(state, dtype='complex')
#         if times is None:
#             t, omega, phase, detuning, dephasing, decoherence = v
#             times = [t]
#         else:
#             omega, phase, detuning, dephasing, decoherence = v

#         H = (omega * _np.cos(phase) * self.Hx + omega * _np.sin(phase) * self.Hy + detuning * self.Hz)
#         L = dephasing * self.dephasing_generator + decoherence * self.decoherence_generator

#         processes = [change_basis(_expm((H + L) * t),
#                                                           'pp', 'col') for t in times]
#         states = [unvec(_np.dot(process, vec(_np.outer(state, state.conj())))) for process in processes]

#         return states

#     def __call__(self, v, times=None, comm=_comm):
#         print(f'Calling process tomography as {comm.Get_rank()} of {comm.Get_size()} on {comm.Get_name()}.')
#         processes = do_process_tomography(self.advance, opt_args={'v':v, 'times':times},
#                                           n_qubits = 1, basis='pp', time_dependent=True, comm=comm)

#         return processes

    
    
# gy = PhysicalProcess(mpi_workers_per_process=1, basis='col')
# gy.set_process_function(ProcessFunction(), mpi_enabled=True)
# target = change_basis(_np.array([[1,0,0,0],
#                                  [0,1,0,0],
#                                  [0,0,0,-1],
#                                  [0,0,1,0]], dtype='complex'), 'pp', 'pp')
# gy.set_target(target)

# # # # Evaluate one time point per run
# # # gy.set_parameter_range([[_np.pi/2-.5, _np.pi/2+.5],[0.9,1.1],[-.1,.1],[-.1,.1],[0,0.1],[0,0.1]])
# # # gy.set_interpolation_order([3,3,3,3,3,3])
# #
# # Evaluate many time points per run
# # Notice that the number of parameters listed below is 1 fewer than in the previous example
# gy.set_parameter_range([[0.9, 1.1], [-.1, .1], [-.1, .1], [0, 0.1], [0, 0.1]])
# gy.set_interpolation_order([3,3,3,3,3])
# gy.set_times(_np.linspace(_np.pi / 2, _np.pi / 2 + .5, 10))

# gy.interpolate()